# pQTL v PD two sample MR

In [ ]:
library("TwoSampleMR")
library("ieugwasr")

# Traits of interest

## Exposure GWAS

### Get the available GWASs in the database

In [ ]:
ao = gwasinfo()
str(ao)

In [ ]:
head(ao)

### Extract the list of pQTLs to lookup

In [ ]:
exp_traits = unique(ao[grepl("prot-a-", ao$id),]$id)
str(exp_traits)

### Example - get descriptive statistics of pQTL

In [ ]:
gwasinfo("prot-a-1363")

## Outcome GWAS

In [ ]:
outcome_traits = c("ukb-b-6548", "ukb-b-956", "ukb-b-16943")
### Double check IDs
for(id in outcome_traits) {
    cat("id: ", id, "\n")
    print(gwasinfo(id))
    cat("\n\n")
}

# Extract the instruments from the pQTL GWAS

In [ ]:
exp.df = tophits(
  id=exp_traits,
  pval=5e-08, # default
  clump=1, # flag to clump or not
  r2=0.001, # default
  kb=10000 # default
)
exp.df

# Find the instruments in the outcome GWASs

In [ ]:
snp_list = unique(exp.df$rsid)
str(snp_list)

In [ ]:
out.df = associations(
  variants=snp_list, #rsids to lookup
  id=outcome_traits, #traits to lookup
  proxies=1, #Look for proxies in outcome.
  r2=0.8, #proxy r2 required
  align_alleles=1, #default
  palindromes=1, #default
  maf_threshold=0.3, #default
)
out.df